Hello! 

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
spark = (SparkSession.builder.config("spark.driver.memory","4g").config("spark.driver.maxResultSize", "4g").getOrCreate())

In [5]:
df = spark.read.csv(path='../input/spotify-charts/charts.csv', inferSchema=True, header=True)

In [6]:
df1 = df.artist

In [7]:
import re

GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)

def simple_word_tokenizer(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]

def token_r(text):
    words = simple_word_tokenizer(text)
    return [x for x in words if len(x) > 1]

In [8]:
conf = SparkConf()

conf.setAll(
    [
        ("spark.app.name", "Word Count"),
        ("spark.master", "local[*]")
    ]
)



In [10]:
count = df1.rdd\
    .flatMap(lambda row: token_r(row.d1))\
    .map(lambda word: (word, 1))\
    .reduceByKey(lambda a, b: a+b)\
    .sortBy(lambda x: x[1], ascending=False)

In [ ]:
count.take(10)

In [ ]:
count.saveAsTextFile('count')

In [ ]:
spark.stop()